Assuming that the images from male and female are in no particular order

In [73]:
import pandas as pd
import dlib
from tqdm import tqdm
import os
import itertools

In [91]:
k = 5     # Number of folds

In [3]:
male_fn = '/home/monash/Desktop/fyp-work/fyp-ma-13/fyp-models/preprocessing/combined_filtered/full/male'
female_fn = '/home/monash/Desktop/fyp-work/fyp-ma-13/fyp-models/preprocessing/combined_filtered/full/female'

In [82]:
def split_into_batches(k, fn):
    """
    Splits dataset into k even batches. Any remaining images are ignored9
    
    """
    folder = os.listdir(fn)
    batch_size = len(folder)//k
    splits = []
    for i in range(k):
        splits.append(folder[i*batch_size:(i+1)*batch_size])
    print("Folder length: ", len(folder))
    print("Batch size: ", batch_size)
    print("Remaining: ", len(folder)%batch_size)
    return splits

In [83]:
male_splits = split_into_batches(k, male_fn)
print()
female_splits = split_into_batches(k, female_fn)

Folder length:  19354
Batch size:  3870
Remaining:  4

Folder length:  22026
Batch size:  4405
Remaining:  1


In [87]:
def chain(lst):
    """
    Chains all elements in a list together into one list
    
    lst: list
        List of lists
    """
    res = []
    for ele in lst:
        res += ele
    return res

def cross_validation(k, splits):
    """
    Perform cross-validation and returns all possible combinations after k-fold cv. 
    Here we take k-2 folds for training and 2 for val/test set

    k : int
        Number of k-folds
    splits: list
        Filenames split into k-folds
    """
    # i and j are the indices of the val/test folds
    i = 0
    res = []
    while i < k-1:
        j = i + 1
        while j < k:
            valTest = [splits[i], splits[j]]
            train = [i for j, i in enumerate(male_splits) if j not in indices]
            res.append([chain(train), chain(valTest)])
            j+=1
        i += 1
    return res

In [90]:
male_cv = cross_validation(k, male_splits)
female_cv = cross_validation(k, female_splits)

In [ ]:
# [training images, val/test images]

for i in male_cv:
    train, valTest = i[0], i[1]
    # Save training images 
    for image_fn in train:
        # Save image
    
    # Split into val (30% of original total) and test (10% of original total)
    batch_size = len(valTest)//4
    
    # Save val images
    for j in range():
        
    # Save test images

